# Title

Intro / research explenation <br>
Link to datasets

In [8]:
import yaml
import pandas as pd

## Load files and create data frames

In [48]:
with open("config.yaml", "r") as stream:
    config_values = yaml.safe_load(stream)
    temperature_path = config_values['temperature_data_path']
    townships_path = config_values['townships_data_path']
    mortality_path = config_values['mortality_data_path']
    
# Create data frames
temperature_df = pd.read_csv(temperature_path, skiprows=11)
townships_df = pd.read_excel(townships_path)
mortality_df = pd.read_csv(mortality_path, sep=';')

## Inspect data

## Wrangle data

## Merge dataframes

## Analyse data (create graphs, etc.)

## Statistics